In [1]:
import numpy as np              ;   import pandas as pd
import geopandas as gpd         ;   from shapely.geometry import Polygon, Point, LineString
import shapely as sp            ;   import matplotlib.pyplot as plt
import time                     ;   import unidecode
import re

startAll = time.time()
def t():
    global start
    start = time.time()
def tt():
    end = time.time() ; minutos = int(round(end - start)/60) ; segundos = int(round(end - start)-(int(round(end - start)/60))*60) ;     tiempo = 'Esta celda tardo en ejecutar: '+str(minutos)+' minutos, '+str(segundos)+' segundos.' ;
    return tiempo
def tiempo():
    endAll = time.time() ; minutos = int(round(endAll - startAll)/60) ; segundos = int(round(endAll - startAll)-(int(round(endAll - startAll)/60))*60) ;     tiempo = 'Desde el inicio, tardo en ejecutar: '+str(minutos)+' minutos, '+str(segundos)+' segundos.' ;
    return tiempo

# PROPERATTI DATABASE

In [3]:
ls

 Volume in drive C is SYSTEM2
 Volume Serial Number is 70F1-0DE5

 Directory of C:\Users\Manu\OneDrive\DH_Dscience\Clase06_Desafio

04/02/2020  05:37 PM    <DIR>          .
04/02/2020  05:37 PM    <DIR>          ..
04/02/2020  12:45 PM    <DIR>          .ipynb_checkpoints
04/02/2020  05:34 PM            10,847 Boxplot.png
03/22/2020  02:40 PM    <DIR>          carto
04/02/2020  05:34 PM        14,522,348 df_limpio.csv
04/02/2020  03:45 PM    <DIR>          documentacion
04/01/2020  08:55 PM           162,119 geo_visualizacion_para juntar.ipynb
04/02/2020  05:34 PM            11,569 Histograma seaborn.png
04/02/2020  05:34 PM            11,196 Histograma.png
04/02/2020  03:34 PM         2,956,329 limpieza_espacial.ipynb
04/02/2020  03:50 PM         3,938,940 nulos_plot_&_m2.ipynb
04/02/2020  04:41 PM    <DIR>          PPT
04/02/2020  05:33 PM         6,886,939 Properati Desaf¡o 1 - Grupo 1.ipynb
04/02/2020  05:32 PM    <DIR>          properati_geoshape
07/05/2018  10:51 AM       149,311

In [2]:
df = pd.read_csv('properatti.csv',sep=',',index_col=0)
df.shape

(121220, 25)

In [3]:
len(df.property_type.unique())

4

In [4]:
df.head(2)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...


# LLAVE Primaria
se encuentra en la direccion URL un codigo embebido que distingue cada registro individualmente

In [9]:
df.properati_url.iloc[0]

'http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inmobiliaria-fedele'

In [11]:
'http://www.properati.com.ar/15bo8_.......'

'http://www.properati.com.ar/15bo8_.......'

In [12]:
def get_codigo(data):
    data = data.strip('http://www.properati.com.ar/')
    return data.split('_')[0]

In [13]:
df['id'] = df.properati_url.map(get_codigo)

In [14]:
len(df.id.unique()) == df.shape[0]

True

In [15]:
df.head(2)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob


In [16]:
df.shape

(121220, 26)

# MISSING DATA

In [17]:
# vemos el total de nulos del dataset
df.isna().sum()[df.isna().sum()>0]

place_name                        23
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
description                        2
image_thumbnail                 3112
dtype: int64

### MISSING DATA AMBIENTES

In [18]:
# vemos el total de nulos del dataset en el campo 'rooms'. Son 73830 valores faltantes en total
df.isna().sum()[df.isna().sum()>0][['rooms']]

rooms    73830
dtype: int64

In [19]:
df.rooms.unique()

array([nan,  1.,  4.,  3.,  2.,  6.,  5., 10.,  7.,  9.,  8., 17., 22.,
       15., 12., 11., 14., 16., 20., 13., 25., 19., 30., 18., 32., 24.,
       31., 21., 29., 27., 23., 28.])

# Ambientes 

### Primero tratamos de extraer información de la columna 'title' del dataset original

In [20]:
# creamos una copia del dataset original que solo tenga la columna 'title'
room2 = df[['title']].copy()
room2.head()

,title
0,2 AMB TIPO CASA SIN EXPENSAS EN PB
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH 3 amb. cfte. reciclado
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [21]:
# renombramos la columna 'title' como 'titulo' para que no se confunda con la del dataset original
room2.columns=['titulo']
room2.head()

,titulo
0,2 AMB TIPO CASA SIN EXPENSAS EN PB
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH 3 amb. cfte. reciclado
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [22]:
#le aplicamos la función unidecode a la columna 'titulo' para dejar el formato en minúscula y sin tildes
room2['titulo']=room2.loc[:,'titulo'].astype(str).apply(unidecode.unidecode).str.lower()
room2.head()

,titulo
0,2 amb tipo casa sin expensas en pb
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,2 amb 3er piso con ascensor apto credito
3,ph 3 amb. cfte. reciclado
4,depto 2 amb al contrafrente zona centro/plaza ...


In [23]:
#aplicamos una regex para capturar en la columna 'titulo' aquellas líneas que contengan una ref. al nro. de ambientes
#al aplicarla fx 'extract' a los dos grupos de captura devuelve los resultados en un nuevo dataframe de dos columnas
cant_amb = room2["titulo"].str.extract("(\d+)\s(amb.)")
cant_amb.head()

,0,1
0,2,amb
1,NaN,NaN
2,2,amb
3,3,amb.
4,2,amb


In [24]:
#unimos el df anterior al df copia del original con la fx concat
room2 = pd.concat([room2, cant_amb], axis=1)
room2.head()

,titulo,0,1
0,2 amb tipo casa sin expensas en pb,2,amb
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN
2,2 amb 3er piso con ascensor apto credito,2,amb
3,ph 3 amb. cfte. reciclado,3,amb.
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb


In [25]:
#renombramos las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room2.columns=['titulo', 'nro_amb', 'txt_amb']

In [26]:
# vemos el total de nulos del dataset room2 de las nuevas columnas 'nro_amb' y 'txt_amb'
room2.isna().sum()[room2.isna().sum()>0]

nro_amb    100330
txt_amb    100330
dtype: int64

In [27]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_amb'. Son str y float.
type(room2.nro_amb.iloc[9999])

str

In [28]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar. Por ej: 9', '8', '15', '10', '11', '4000', '360', '12', '14'
# Es probable que no sean ambientes.
room2.nro_amb.unique()

array(['2', nan, '3', '4', '6', '1', '5', '7', '9', '8', '15', '10', '11',
       '4000', '360', '12', '14'], dtype=object)

In [29]:
# repetimos el procedimiento anterior para obtener del título aquellas líneas que contengan una ref. al nro. de dormitorios
cant_dorm = room2["titulo"].str.extract("(\d+)\s(dorm.)")
cant_dorm.head()

,0,1
0,NaN,NaN
1,2,dorm.
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [30]:
#unimos el df anterior al df copia del original con la fx concat
room2 = pd.concat([room2, cant_dorm], axis=1)
room2.head()

,titulo,nro_amb,txt_amb,0,1
0,2 amb tipo casa sin expensas en pb,2,amb,NaN,NaN
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN,2,dorm.
2,2 amb 3er piso con ascensor apto credito,2,amb,NaN,NaN
3,ph 3 amb. cfte. reciclado,3,amb.,NaN,NaN
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,NaN,NaN


In [31]:
#renombramos las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room2.columns=['titulo', 'nro_amb', 'txt_amb', 'nro_dorm', 'txt_dorm']

In [32]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_dorm'. Son str y float.
type(room2.nro_dorm.iloc[1])

str

In [33]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar. Por ej: '7', '6', '10', '9'
# Es probable que no sean ambientes.
room2.nro_dorm.unique()

array([nan, '2', '5', '1', '3', '4', '7', '6', '10', '9'], dtype=object)

In [34]:
# vemos el total de nulos del dataset room2. Hay menos referencias a dormitorios que a ambientes.
room2.isna().sum()[room2.isna().sum()>0]

nro_amb     100330
txt_amb     100330
nro_dorm    114056
txt_dorm    114056
dtype: int64

In [35]:
# repetimos el procedimiento anterior para obtener del título aquellas líneas que contengan una ref. al nro. de habitaciones
cant_hab = room2["titulo"].str.extract("(\d+)\s(habitac...)")
cant_hab

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
121215,NaN,NaN
121216,NaN,NaN
121217,NaN,NaN
121218,NaN,NaN


In [36]:
#unimos el df anterior al df copia del original con la fx concat
room2 = pd.concat([room2, cant_hab], axis=1)
room2.head()

,titulo,nro_amb,txt_amb,nro_dorm,txt_dorm,0,1
0,2 amb tipo casa sin expensas en pb,2,amb,NaN,NaN,NaN,NaN
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN,2,dorm.,NaN,NaN
2,2 amb 3er piso con ascensor apto credito,2,amb,NaN,NaN,NaN,NaN
3,ph 3 amb. cfte. reciclado,3,amb.,NaN,NaN,NaN,NaN
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,NaN,NaN,NaN,NaN


In [37]:
#renombramos nuevamente las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room2.columns=['titulo', 'nro_amb', 'txt_amb', 'nro_dorm', 'txt_dorm', 'nro_hab', 'txt_hab']
room2.head()

,titulo,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab
0,2 amb tipo casa sin expensas en pb,2,amb,NaN,NaN,NaN,NaN
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN,2,dorm.,NaN,NaN
2,2 amb 3er piso con ascensor apto credito,2,amb,NaN,NaN,NaN,NaN
3,ph 3 amb. cfte. reciclado,3,amb.,NaN,NaN,NaN,NaN
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,NaN,NaN,NaN,NaN


In [38]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_amb'. Son str y float.
type(room2.nro_hab.iloc[10])

float

In [39]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar. Por ej: '14', '20', '9', '16', '21', '27', '11', '10'
# Es probable que no sean ambientes.
room2.nro_hab.unique()

array([nan, '2', '1', '8', '3', '4', '14', '20', '9', '5', '16', '21',
       '27', '11', '10'], dtype=object)

In [40]:
# vemos el total de nulos del dataset room2. Las referencias a las habitaciones son menos que a las de dormitorios
room2.isna().sum()[room2.isna().sum()>0]

nro_amb     100330
txt_amb     100330
nro_dorm    114056
txt_dorm    114056
nro_hab     121150
txt_hab     121150
dtype: int64

In [41]:
#Rellenamos todos los NaN con ''
room2.nro_amb.fillna("",inplace=True)
room2.txt_amb.fillna("",inplace=True)
room2.nro_dorm.fillna("",inplace=True)
room2.txt_dorm.fillna("",inplace=True)
room2.nro_hab.fillna("",inplace=True)
room2.txt_hab.fillna("",inplace=True)
room2.head()

,titulo,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab
0,2 amb tipo casa sin expensas en pb,2,amb,,,,
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,,,2,dorm.,,
2,2 amb 3er piso con ascensor apto credito,2,amb,,,,
3,ph 3 amb. cfte. reciclado,3,amb.,,,,
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,,,,


In [42]:
#verifico que no haya NaN en las columnas y que haya valores vacíos en su lugar
room2.nro_amb.unique()

array(['2', '', '3', '4', '6', '1', '5', '7', '9', '8', '15', '10', '11',
       '4000', '360', '12', '14'], dtype=object)

In [43]:
type(room2.nro_amb[1])

str

In [44]:
#hacemos un conteo real de nulos después de reemplazar texto por números. 
# Tenemos 94544 nulos.
len(room2.loc[(room2.nro_amb=="")&(room2.nro_dorm=="")&(room2.nro_hab=="")])

93189

In [45]:
#creamos una nueva columna que es la suma de ambientes, dormitorios y habitaciones.
#Gran supuesto: si hay ref a cantidad de ambientes cubiertos, no la hay de dormitorios o habitaciones.
#por lo tanto 'ambientes' reúne la cantidad de ambientes que pudimos extraer de 'titulo'
room2['ambientes_tit'] = room2['nro_amb'] + room2['nro_dorm'] + room2['nro_hab']
room2.head(15)

,titulo,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab,ambientes_tit
0,2 amb tipo casa sin expensas en pb,2,amb,,,,,2
1,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,,,2,dorm.,,,2
2,2 amb 3er piso con ascensor apto credito,2,amb,,,,,2
3,ph 3 amb. cfte. reciclado,3,amb.,,,,,3
4,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,,,,,2
5,"casa barrio 338. sobre calle 3 de caballeria, ...",,,,,,,
6,"muy buen ph al frente dos dormitorios , patio,...",,,,,,,
7,jose hernandez 1400 monoambiente estrenar cat...,,,,,,,
8,"jose hernandez 1400 dos ambientes estrenar ,...",,,,,,,
9,white 7637 - 2 dormitorios con patio,,,2,dormi,,,2


In [46]:
#vemos que son todos str
type(room2['ambientes_tit'][1000])

str

In [47]:
room2.ambientes_tit.unique()

array(['2', '3', '', '4', '5', '6', '1', '43', '32', '52', '7', '9', '8',
       '53', '15', '64', '54', '21', '14', '22', '10', '11', '42', '20',
       '23', '24', '65', '13', '4000', '16', '27', '3601', '12', '84'],
      dtype=object)

In [48]:
#reemplazo los str que son valores atípicos
room2.ambientes_tit=room2.ambientes_tit.str.replace('43','').replace('32','').replace('52','').replace('53','').replace('64','').replace('54','').replace('21','').replace('42','').replace('20','').replace('23','').replace('24','').replace('22','').replace('65','').replace('4000','').replace('27','').replace('3601','').replace('84','')

In [49]:
room2.ambientes_tit.unique()

array(['2', '3', '', '4', '5', '6', '1', '7', '9', '8', '15', '14', '10',
       '11', '13', '16', '12'], dtype=object)

In [50]:
len(room2.loc[(room2.ambientes_tit=="")])

93285

In [51]:
room2.dtypes

titulo           object
nro_amb          object
txt_amb          object
nro_dorm         object
txt_dorm         object
nro_hab          object
txt_hab          object
ambientes_tit    object
dtype: object

In [52]:
#concatenamos el room2 al df original
df = pd.concat([df, room2], axis=1)
df.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,image_thumbnail,id,titulo,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab,ambientes_tit
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2 amb tipo casa sin expensas en pb,2,amb,,,,,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,venta depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,,,2,dorm.,,,2
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2 amb 3er piso con ascensor apto credito,2,amb,,,,,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,ph 3 amb. cfte. reciclado,3,amb.,,,,,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,depto 2 amb al contrafrente zona centro/plaza ...,2,amb,,,,,2


In [53]:
#quitamos las columnas que no nos interesan
df=df.drop (["titulo","nro_amb","txt_amb","nro_dorm","txt_dorm", "nro_hab","txt_hab"], axis=1)
df.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2


In [54]:
#chequeamos que la cantidad de registros en la col 'rooms' no es igual a la cant de registros en 'ambientes'
print('¿Cantidad total calculada y original son iguales? {}'\
       .format(df.ambientes_tit.equals(df.rooms)))

¿Cantidad total calculada y original son iguales? False


In [55]:
#creamos una máscara booleana que nos dice 'True' los que no son iguales y 'False' los que coinciden
(df.ambientes_tit != df.rooms)

0         True
1         True
2         True
3         True
4         True
          ... 
121215    True
121216    True
121217    True
121218    True
121219    True
Length: 121220, dtype: bool

### En segundo lugar tratamos de extraer cantidad de ambientes de la columna "description" 

In [56]:
#copiamos la columna 'description' a un dataframe distinto de una columna
room3 = df[['description']].copy()
room3.head()

,description
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,..."
1,Venta de departamento en décimo piso al frente...
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3,PH 3 ambientes con patio. Hay 3 deptos en lote...
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...


In [57]:
# renombramos la columna 'descrption' como 'descrip' para que no se confunda con la del dataset original
room3.columns=['descrip']
room3.head()

,descrip
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,..."
1,Venta de departamento en décimo piso al frente...
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3,PH 3 ambientes con patio. Hay 3 deptos en lote...
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...


In [58]:
#le aplicamos la función unidecode a la columna 'descrip' para dejar el formato en minúscula y sin tildes
room3['descrip']=room3.loc[:,'descrip'].astype(str).apply(unidecode.unidecode).str.lower()
room3.head()

,descrip
0,"2 ambientes tipo casa planta baja por pasillo,..."
1,venta de departamento en decimo piso al frente...
2,2 ambientes 3er piso lateral living comedor am...
3,ph 3 ambientes con patio. hay 3 deptos en lote...
4,departamento con fantastica iluminacion natura...


In [59]:
#aplicamos una regex para capturar en la columna 'titulo' aquellas líneas que contengan una ref. al nro. de ambientes
#al aplicarla fx 'extract' a los dos grupos de captura devuelve los resultados en un nuevo dataframe de dos columnas
cant_amb_descrip = room3["descrip"].str.extract("(\d+)\s(amb.)")
cant_amb_descrip.head()

,0,1
0,2,ambi
1,NaN,NaN
2,2,ambi
3,3,ambi
4,NaN,NaN


In [60]:
#unimos el df anterior al df copia del original con la fx concat
room3 = pd.concat([room3, cant_amb_descrip], axis=1)
room3.head()

,descrip,0,1
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi
1,venta de departamento en decimo piso al frente...,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi
4,departamento con fantastica iluminacion natura...,NaN,NaN


In [61]:
#renombramos las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room3.columns=['descrip', 'nro_amb', 'txt_amb']
room3.head()

,descrip,nro_amb,txt_amb
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi
1,venta de departamento en decimo piso al frente...,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi
4,departamento con fantastica iluminacion natura...,NaN,NaN


In [62]:
# vemos el total de nulos del dataset room2 de las nuevas columnas 'nro_amb' y 'txt_amb'
room3.isna().sum()[room3.isna().sum()>0]

nro_amb    83681
txt_amb    83681
dtype: int64

In [63]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_amb'. Son str y float.
type(room3.nro_amb.iloc[15])

float

In [64]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar.
# Es probable que no sean ambientes.
room3.nro_amb.unique()

array(['2', nan, '3', '1', '4', '5', '7', '6', '415', '50', '042', '8',
       '25', '9', '23', '2017', '103', '297', '42026', '072', '902', '18',
       '000', '11', '944', '22', '0002', '153', '472', '281', '442', '24',
       '22330489662', '10', '12', '20173', '9502', '1200', '33', '772',
       '652', '004', '653', '983', '42025', '42004', '42028', '151',
       '42046', '42049', '412', '42015', '42021', '172', '28612', '20174',
       '547', '693', '622', '733', '14', '183', '793', '00', '253', '006',
       '913', '20171', '20172', '853', '173', '162', '123', '0', '202',
       '192', '113', '17', '90', '0001', '2004', '20', '822', '562',
       '42054', '293', '65', '84', '872', '164', '312', '42024', '102'],
      dtype=object)

In [65]:
# repetimos el procedimiento anterior para obtener del título aquellas líneas que contengan una ref. al nro. de dormitorios
cant_dorm_descrip = room3["descrip"].str.extract("(\d+)\s(dorm.)")
cant_dorm_descrip.head()

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,1,dormi


In [66]:
#unimos el df anterior al df copia del original con la fx concat
room3 = pd.concat([room3, cant_dorm_descrip], axis=1)
room3.head()

,descrip,nro_amb,txt_amb,0,1
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,NaN,NaN
1,venta de departamento en decimo piso al frente...,NaN,NaN,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,NaN,NaN
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,NaN,NaN
4,departamento con fantastica iluminacion natura...,NaN,NaN,1,dormi


In [67]:
#renombramos nuevamente las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room3.columns=['descrip', 'nro_amb', 'txt_amb', 'nro_dorm', 'txt_dorm']
room3.head()

,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,NaN,NaN
1,venta de departamento en decimo piso al frente...,NaN,NaN,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,NaN,NaN
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,NaN,NaN
4,departamento con fantastica iluminacion natura...,NaN,NaN,1,dormi


In [68]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_dorm'. Son str y float.
type(room3.nro_dorm.iloc[0])

float

In [69]:
# vemos el total de nulos del dataset room2. Hay más referencias a dormitorios que a ambientes.
room3.isna().sum()[room3.isna().sum()>0]

nro_amb     83681
txt_amb     83681
nro_dorm    80423
txt_dorm    80423
dtype: int64

In [70]:
# repetimos el procedimiento anterior para obtener del título aquellas líneas que contengan una ref. al nro. de habitaciones
cant_hab_descrip = room3["descrip"].str.extract("(\d+)\s(hab...)")
cant_hab_descrip.head()

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [71]:
#unimos el df anterior al df copia del original con la fx concat
room3 = pd.concat([room3, cant_hab_descrip], axis=1)
room3.head()

,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm,0,1
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,NaN,NaN,NaN,NaN
1,venta de departamento en decimo piso al frente...,NaN,NaN,NaN,NaN,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,NaN,NaN,NaN,NaN
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,NaN,NaN,NaN,NaN
4,departamento con fantastica iluminacion natura...,NaN,NaN,1,dormi,NaN,NaN


In [72]:
#renombramos nuevamente las nuevas columnas para quitarnos de encima los nombres '0' y '1' que se llamaban por default 
room3.columns=['descrip', 'nro_amb', 'txt_amb', 'nro_dorm', 'txt_dorm', 'nro_hab', 'txt_hab']
room3.head()

,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,NaN,NaN,NaN,NaN
1,venta de departamento en decimo piso al frente...,NaN,NaN,NaN,NaN,NaN,NaN
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,NaN,NaN,NaN,NaN
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,NaN,NaN,NaN,NaN
4,departamento con fantastica iluminacion natura...,NaN,NaN,1,dormi,NaN,NaN


In [73]:
# vemos qué tipo de objeto es cada valor de la columna 'nro_amb'. Son str y float.
type(room3.nro_hab.iloc[10])

float

In [74]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar. Por ej: '14', '20', '9', '16', '21', '27', '11', '10'
# Es probable que no sean ambientes.
room3.nro_hab.unique()

array([nan, '2', '3', '2000', '4', '5', '30', '1', '9', '55', '6', '8',
       '16', '290', '7', '10', '13', '400', '800', '14', '54', '1500',
       '21', '57', '24', '12', '140', '71', '72', '130', '425', '90',
       '120', '200', '11', '150', '600', '27', '20', '170', '51', '160',
       '40', '047', '80', '17', '103', '450', '02', '102', '66', '18',
       '16002', '210', '000', '22', '124', '25', '29', '23', '52', '33',
       '15', '32', '50', '1000', '56', '500', '36'], dtype=object)

In [75]:
# vemos el total de nulos del dataset room2. Las referencias a las habitaciones son menos que a las de dormitorios
room3.isna().sum()[room3.isna().sum()>0]

nro_amb      83681
txt_amb      83681
nro_dorm     80423
txt_dorm     80423
nro_hab     118161
txt_hab     118161
dtype: int64

In [76]:
#Rellenamos todos los NaN con ''
room3.nro_amb.fillna("",inplace=True)
room3.txt_amb.fillna("",inplace=True)
room3.nro_dorm.fillna("",inplace=True)
room3.txt_dorm.fillna("",inplace=True)
room3.nro_hab.fillna("",inplace=True)
room3.txt_hab.fillna("",inplace=True)
room3.head()

,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,,,,
1,venta de departamento en decimo piso al frente...,,,,,,
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,,,,
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,,,,
4,departamento con fantastica iluminacion natura...,,,1,dormi,,


In [77]:
#hacemos un conteo real de nulos después de reemplazar texto por números. 
# Tenemos 50507 nulos.
len(room3.loc[(room3.nro_amb=="")&(room3.nro_dorm=="")&(room3.nro_hab=="")])

50492

In [78]:
#creamos una nueva columna que es la suma de ambientes, dormitorios y habitaciones.
#Gran supuesto: si hay ref a cantidad de ambientes cubiertos, no la hay de dormitorios o habitaciones.
#por lo tanto 'ambientes' reúne la cantidad de ambientes que pudimos extraer de 'titulo'
room3['ambientes_descrip'] = room3['nro_amb'] + room3['nro_dorm'] + room3['nro_hab']
room3.head(50)

,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab,ambientes_descrip
0,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,,,,,2
1,venta de departamento en decimo piso al frente...,,,,,,,
2,2 ambientes 3er piso lateral living comedor am...,2,ambi,,,,,2
3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,,,,,3
4,departamento con fantastica iluminacion natura...,,,1,dormi,,,1
5,"casa en el perimetro del barrio 338, ubicada e...",,,,,,,
6,muy buen ph al frente con entrada independient...,,,,,,,
7,excelente monoambiente a estrenar amplio super...,,,,,,,
8,excelente dos ambientes estrenar amplio super...,,,,,,,
9,mednoza al 7600a una cuadra de calle mendozawh...,,,2,dormi,,,2


In [79]:
#Corremos una fx unique para ver cuáles son valores que se levantaron. 
# Hay valores atípicos a revisar. Es probable que no sean ambientes.
room3.ambientes_descrip.unique()

array(['2', '', '3', '1', '12', '230', '43', '5', '32', '22', '4', '54',
       '53', '74', '42000', '21', '42', '24', '23', '31', '63', '6', '44',
       '75', '30', '51', '64', '33', '52', '35', '62', '45', '72', '8',
       '11', '432', '4154', '503', '41', '9', '7', '322', '345', '55',
       '103', '9504', '228', '48', '042', '84', '493', '73', '2516', '93',
       '1290', '232', '184', '77', '10', '18', '13', '400', '340', '34',
       '43800', '2017', '61', '380', '297', '42026', '072', '14', '65',
       '1500', '350', '902', '183', '102', '0002', '742', '57', '622',
       '710', '714', '7521', '944', '511', '76', '221', '60', '2140',
       '88', '99', '278', '532', '412', '71', '1532', '87', '22130',
       '2425', '365003', '472', '321', '002', '290', '120', '422', '722',
       '83', '281', '612', '20', '00', '420', '442', '241', '22330489662',
       '2200', '542', '15111', '11491', '29271', '9311', '13303', '6731',
       '18472', '24161', '15642', '9152', '16801', '2679

In [80]:
#concatenamos el room3 al df original
df = pd.concat([df, room3], axis=1)
df.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,id,ambientes_tit,descrip,nro_amb,txt_amb,nro_dorm,txt_dorm,nro_hab,txt_hab,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,15bo8,2,"2 ambientes tipo casa planta baja por pasillo,...",2,ambi,,,,,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,15bob,2,venta de departamento en decimo piso al frente...,,,,,,,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,15bod,2,2 ambientes 3er piso lateral living comedor am...,2,ambi,,,,,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,15boh,3,ph 3 ambientes con patio. hay 3 deptos en lote...,3,ambi,,,,,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,15bok,2,departamento con fantastica iluminacion natura...,,,1,dormi,,,1


In [81]:
#quitamos las columnas que no nos interesan
df=df.drop (["descrip","nro_amb","txt_amb","nro_dorm","txt_dorm", "nro_hab","txt_hab"], axis=1)
df.head(12)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,NaN,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [82]:
#completamos los valores faltantes NaN con los valores que sacamos de la columna título, que llamamos 'ambientes_tit'
df['rooms'] = df['rooms'].fillna(df['ambientes_tit'])
df.head(12)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [83]:
#Ahora tenemos 62053 nulos que hay que depurar
len(df.loc[df.rooms == ''])


62053

In [84]:
# vemos que en todos los valores hay valores atípicos tanto str como float que hay que limpiar
df.rooms.unique()

array(['2', '3', '', 1.0, 4.0, 3.0, 2.0, 6.0, 5.0, '6', '1', '4', 10.0,
       7.0, 9.0, 8.0, '5', 17.0, 22.0, 15.0, '9', 12.0, 11.0, '7', 14.0,
       16.0, 20.0, '8', 13.0, '15', 25.0, 19.0, '10', 30.0, 18.0, 32.0,
       '11', '16', '12', '14', 24.0, 31.0, 21.0, 29.0, 27.0, 23.0, 28.0],
      dtype=object)

In [85]:
#Transformo a float todos los valores nuevamente
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')

In [86]:
df.head(20)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2.0,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2.0,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2.0,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3.0,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2.0,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2.0,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [87]:
#veo cuales son los valores que tiene mi columna 'rooms'
df.rooms.unique()

array([ 2.,  3., nan,  1.,  4.,  6.,  5., 10.,  7.,  9.,  8., 17., 22.,
       15., 12., 11., 14., 16., 20., 13., 25., 19., 30., 18., 32., 24.,
       31., 21., 29., 27., 23., 28.])

In [88]:
#ahora rellenamos los NaN con los valores de los ambientes que extraimos de la descripción incluidos en la columna 'ambientes_descrip'
df['rooms'] = df['rooms'].fillna(df['ambientes_descrip'])
df.head(20)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [89]:
#vemos que al agregar nuevos registros, tenemos valores atípicos. También tenemos tanto floats como str
df['rooms'].unique()

array([2.0, 3.0, '', '3', '12', 1.0, 4.0, 6.0, '2', 5.0, '1', '4', '32',
       10.0, 7.0, 9.0, 8.0, '74', '5', '24', '43', '63', '6', '53', 17.0,
       22.0, '52', '21', '35', '62', '31', 15.0, '11', '54', '9', '23',
       '7', '42', '48', 12.0, '73', 11.0, 14.0, 16.0, '93', '22', 20.0,
       '77', '33', 13.0, '34', '43800', '2017', '103', '41', '297', '072',
       '64', '102', '742', '57', '622', '61', '714', '511', '8', '2140',
       25.0, '65', '10', '532', '51', '30', '22130', '365003', '44', 19.0,
       '422', '72', '88', '109', '330', '150', '600', '652', '610', '13',
       '421', '83', '631', '542', '1042', '27', '20', 30.0, '170', '87',
       '94', '71', '632', '26', '75', '323', 18.0, '711', '403', '14',
       '621', '00022', 32.0, '23001', '17', '100', '201741', '80',
       '42181824', '113', '613', '18', '3204', '92', '222', '342', '5210',
       '29', '125', '2000', '1000', '124', '55', '233', '485', '000',
       '00', '25', '90', '50', '9014', '84', '0001', '90

In [90]:
#previo a limpiar los valores atípicos tenemos 31674 registros
len(df.loc[df.rooms == ''])

31674

In [91]:
#Probamos como limpiar un str..
df.rooms=df.rooms.replace('12','')

In [92]:
df.head(12)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [93]:
type(df.rooms.iloc[11])

str

In [94]:
#reemplazamos los str y floats para todos los valores atípicos
df.rooms=df.rooms.replace('32',3.0).replace('74','').replace('24','').replace('43',4.0)
df.rooms=df.rooms.replace('63',6.0).replace('53',5.0).replace('52',5.0).replace('21',2.0).replace('35',3.0)
df.rooms=df.rooms.replace('62','').replace('31',3.0).replace('11','').replace('54',5.0).replace('23','')
df.rooms=df.rooms.replace('42',4.0).replace('48','').replace('73','').replace('93','').replace('22',2.0)
df.rooms=df.rooms.replace('33',3.0).replace('34',3.0).replace('43800','').replace('2017','').replace('103','')
df.rooms=df.rooms.replace('41','').replace('297','').replace('072','').replace('64','6').replace('102','')
df.rooms=df.rooms.replace('742','').replace('57','').replace('622','').replace('61','6').replace('714','')
df.rooms=df.rooms.replace('511',5.0).replace('2140','').replace('65','').replace('532','').replace('51','')
df.rooms=df.rooms.replace('30',3.0).replace('22130','').replace('365003','').replace('44',4.0).replace('422',4.0)
df.rooms=df.rooms.replace('72','').replace('88','').replace('109','').replace('330',3.0).replace('150','')
df.rooms=df.rooms.replace('600','').replace('652','').replace('610','').replace('13','').replace('421',4.0)
df.rooms=df.rooms.replace('83','').replace('631','').replace('26','').replace('75','').replace('323','')
df.rooms=df.rooms.replace('711','').replace('403','').replace('14','').replace('621','').replace('00022','')
df.rooms=df.rooms.replace('23001','').replace('100','').replace('17','').replace('100','').replace('201741','')
df.rooms=df.rooms.replace('42181824','').replace('113','').replace('613','').replace('18','').replace('3204','')
df.rooms=df.rooms.replace('92','').replace('222',2.0).replace('342','').replace('5210','').replace('29','')
df.rooms=df.rooms.replace('125','').replace('2000','').replace('1000','').replace('124','').replace('55',5.0)
df.rooms=df.rooms.replace('233','').replace('485','').replace('000','').replace('00','').replace('25','')
df.rooms=df.rooms.replace('90','').replace('50','').replace('9014','').replace('84','').replace('0001','')
df.rooms=df.rooms.replace('902','').replace('772','').replace('140','').replace('350','').replace('117','')
df.rooms=df.rooms.replace('5001','').replace('141','').replace('47','').replace('400','').replace('60','')
df.rooms=df.rooms.replace('2',2.0).replace('644','').replace('5104','').replace('662','').replace('431','')
df.rooms=df.rooms.replace('82','').replace('45','').replace('423','').replace('315','').replace('322',3.0)
df.rooms=df.rooms.replace('4000','').replace('326','').replace('98','').replace('76','').replace('85','')
df.rooms=df.rooms.replace('1500','').replace('2170','').replace('3170','').replace('148','').replace('15','')
df.rooms=df.rooms.replace('6003','').replace('432','').replace('602','').replace('28','').replace('19754','')
df.rooms=df.rooms.replace('232','').replace('66','').replace('722','').replace('05','5').replace('56','')
df.rooms=df.rooms.replace('20173','').replace('36','').replace('5802','').replace('104','')

In [95]:
df.head(12)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [96]:
# vemos con qué valores nos quedamos 
df['rooms'].unique()

array([2.0, 3.0, '', '3', 1.0, 4.0, 6.0, 5.0, '1', '4', 10.0, 7.0, 9.0,
       8.0, '5', '6', 17.0, 22.0, 15.0, '9', '7', 12.0, 11.0, 14.0, 16.0,
       20.0, '77', 13.0, '8', 25.0, '10', 19.0, '542', '1042', '27', '20',
       30.0, '170', '87', '94', '71', '632', 18.0, 32.0, '80', '211',
       24.0, 31.0, 21.0, 29.0, '218', 27.0, 23.0, '40', 28.0],
      dtype=object)

In [97]:
#repetimos nuevamente para ver cuántos son los valores que tenemos
len(df.loc[df.rooms == ''])

32184

In [98]:
#limpiamos los valores atípicos que nos faltaban
df.rooms=df.rooms.replace('3',3.0).replace('1',1.0).replace('4',4.0).replace('5',5.0).replace('6',6.0)
df.rooms=df.rooms.replace(17.0,'').replace(22.0,'').replace(15.0,'').replace('9',9.0).replace(12.0,'')
df.rooms=df.rooms.replace(11.0,'').replace(14.0,'').replace(16.0,'').replace(20.0,'').replace('77','')
df.rooms=df.rooms.replace(13.0,'').replace('8',8.0).replace(25.0,'').replace('10',10.0).replace(19.0,'')
df.rooms=df.rooms.replace('542','').replace('1042','').replace('27','').replace('20','')
df.rooms=df.rooms.replace(30.0,'').replace('170','').replace('87','').replace('94','')
df.rooms=df.rooms.replace('71','').replace('632','').replace(18.0,'').replace(32.0,'')
df.rooms=df.rooms.replace('80','').replace('211','').replace(24.0,'').replace(31.0,'')
df.rooms=df.rooms.replace(21.0,'').replace(29.0,'').replace('218','').replace(27.0,'').replace(23.0,'')
df.rooms=df.rooms.replace('40','').replace(28.0,'').replace('7',7.0)

In [99]:
#vemos que todos los valores son floats
df['rooms'].unique()

array([2.0, 3.0, '', 1.0, 4.0, 6.0, 5.0, 10.0, 7.0, 9.0, 8.0],
      dtype=object)

In [100]:
#nos quedamos con 32522 registros de ambientes para la columna 'rooms'
len(df.loc[df.rooms == ''])

32522

In [101]:
#ahora transformamos los '' en NaN para que toda la columna 'rooms' vuelva a tener valores Floats únicamente
df.rooms=df.rooms.replace('',np.NaN)

In [102]:
#y chequeamos que la col 'rooms' efectivamente tiene todos Floats
df.head(12)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,id,ambientes_tit,ambientes_descrip
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,2.0,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,15bo8,2,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,2.0,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,15bob,2,
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,2.0,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,15bod,2,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,3.0,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,15boh,3,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,2.0,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,15bok,2,1
5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,15bop,,
6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,15bor,,
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,15bot,,
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,15bou,,
9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,2.0,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,15box,2,2


In [103]:
df['rooms'].unique()

array([ 2.,  3., nan,  1.,  4.,  6.,  5., 10.,  7.,  9.,  8.])

In [104]:
df.loc[7367]

operation                                                                  sell
property_type                                                             house
place_name                                                coordenadas 34.255511
place_with_parent_names       |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|coor...
country_name                                                          Argentina
state_name                                             Bs.As. G.B.A. Zona Norte
geonames_id                                                                 NaN
lat-lon                                           -34.4284852743,-58.6017169869
lat                                                                    -34.4285
lon                                                                    -58.6017
price                                                                    550000
currency                                                                    USD
price_aprox_local_currency              